# Introduction
In this notebook we will evaluate the models for forecasting the demand of retail products. The best performing model will be deployed to power the demand forecasting dashboard for <a href = "https://walmart.converge.tech/content/converge/en_in/sparkathon.html"><b>Sparathon Challange</b></a> solution we are building.

The data to train and evaluate the forecasting models is taken from a <a href = "https://www.kaggle.com/competitions/m5-forecasting-accuracy/overview"><b>kaggle challange</b></a> hosted by University of Nicosia. The dataset consists of following files:
<ul>
    <li>calendar.csv : Contains the information about dates on which products are sold.</li>
    <li>sales_train_validation.csv : Contains the historical daily unit sales data per product and store [d_1 - d_1913]</li>
    <li>sell_prices.csv : Contains information about the price of the products sold per store and date.</li>
    <li>sales_train_evaluation.csv - Includes sales [d_1 - d_1941]</li>
</ul>

<u><b>Note</b></u> : The demand volume is proxied with the sales value of the products in the data for the purposes of our solution.

In [1]:
# Library Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
# Notebook Runtime Parameters
data_directory = "./data/"
calendar_file_name = "calendar.csv"
sales_evaluation_file_name = "sales_train_evaluation.csv"
sales_validation_file_name = "sales_train_validation.csv"
# historical_sell_prices_file_name = "sell_prices.csv"

In [3]:
# Reading the data from the input CSV files.
calendar_data = pd.read_csv(data_directory + calendar_file_name)
sales_evaluation_data = pd.read_csv(data_directory + sales_evaluation_file_name)
sales_validation_data = pd.read_csv(data_directory + sales_validation_file_name)
# sell_prices_data = pd.read_csv(data_directory + historical_sell_prices_file_name)

# Reduce Memory Usage

In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

sales_evaluation_data = reduce_mem_usage(sales_evaluation_data)
sales_validation_data = reduce_mem_usage(sales_validation_data)

Memory usage of dataframe is 452.91 MB
Memory usage after optimization is: 96.30 MB
Decreased by 78.7%
Memory usage of dataframe is 446.40 MB
Memory usage after optimization is: 95.16 MB
Decreased by 78.7%


In [5]:
# Inspecting Volume of data in each files.
print("Calendar Data : ", calendar_data.shape)
print("Evaluation Data : ", sales_evaluation_data.shape)
print("Validation Data : ", sales_validation_data.shape)
# print("Sell Prices Data : ", sell_prices_data.shape)

Calendar Data :  (1969, 14)
Evaluation Data :  (30490, 1947)
Validation Data :  (30490, 1919)


In [6]:
calendar_data.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [7]:
# sell_prices_data.head()

In [8]:
sales_validation_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [9]:
sales_evaluation_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [10]:
# sell_prices_data["store_id"].value_counts()

# Transform (Wide to Long format)
The sales validation and evaluation data is transformed such that data is in row-event format.

In [11]:
sales_validation_data = pd.melt(sales_validation_data, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'd', value_name = 'sales')
sales_evaluation_data = pd.melt(sales_evaluation_data, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'd', value_name = 'sales')

# Adding Calendar information
The date information related to product sales is added. 

In [12]:
sales_validation_data = sales_validation_data.merge(calendar_data, on = "d", how = "left")
sales_evaluation_data = sales_evaluation_data.merge(calendar_data, on = "d", how = "left")

# Concatenate Data
Concatenated the datasets for harmonized of view of data. The train-test split will be made in out-of-time manner. 

In [13]:
data = pd.concat([sales_validation_data, sales_evaluation_data], axis = 0, ignore_index = True)

In [14]:
del sales_validation_data
del sales_evaluation_data

# Lag based and Moving Window Average Features  

In [ ]:
def create_features(dt, lags = [7, 14, 28], wins = [7, 14, 28]):
    lag_cols = [f"lag_{lag}" for lag in lags]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag).fillna(-1)

    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean()).fillna(-1)
        
    return dt

data = create_features(data)

# Time based Features

In [ ]:
data['date'] = pd.to_datetime(data["date"])

date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",

    }
    
for date_feat_name, date_feat_func in date_features.items():
    if date_feat_name in data.columns:
        data[date_feat_name] = data[date_feat_name].astype("int16")
    else:
        data[date_feat_name] = getattr(data["date"].dt, date_feat_func).astype("int16")

# Label encoding ID columns

In [ ]:
# encode categorical features
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id']
for cc in cat_feats:
    le = LabelEncoder()
    data[cc] = le.fit_transform(data[cc])

# Train-Validation Split

In [ ]:
# split into train and test
cutoff = data.date.max() - pd.to_timedelta(7, unit = 'D')
xtrain = data.loc[data.date < cutoff].copy()
xeval = data.loc[data.date >= cutoff].copy()

# Training the model

In [ ]:
drop_columns = []
feature_columns = list(set(xtrain.columns).difference(set(drop_columns)))
target_column = "sales"

X_train = xtrain[feature_columns]
Y_train = xtrain[target_column]

X_eval = xeval[feature_columns]
Y_eval = xeval[target_column]

In [ ]:
# Model code ...